In [39]:
# Tiktoken is an open-source Python library that breaks text into tokens
#Langchian_community sued to make returiever
#Used chromaDB for database

!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [40]:
import langchain
langchain.__version__

'0.3.20'

In [ ]:
#create .env and add OPENAI_API_KEY, LANGCHAIN_API_KEY

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [42]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [43]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
med_llm = llm.invoke("Tell me how are reference values calculated in lab test reports")
med_llm

AIMessage(content='Reference values in lab test reports are typically calculated based on a large sample of healthy individuals. These individuals are usually selected to be representative of the general population in terms of age, sex, and other relevant factors. The reference values are then determined by calculating the range of values that encompass the majority of these healthy individuals.\n\nIn some cases, reference values may also be adjusted based on specific factors such as age, sex, or ethnicity. This is done to account for variations in normal values that may occur in different populations.\n\nIt is important to note that reference values are not set in stone and may vary slightly between different laboratories and testing methods. Additionally, reference values are meant to be used as a general guide and should be interpreted in conjunction with other clinical information to make an accurate diagnosis.', additional_kwargs={'refusal': None}, response_metadata={'token_usage'

In [44]:
#OutputParser for getting only the output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(med_llm)

'Reference values in lab test reports are typically calculated based on a large sample of healthy individuals. These individuals are usually selected to be representative of the general population in terms of age, sex, and other relevant factors. The reference values are then determined by calculating the range of values that encompass the majority of these healthy individuals.\n\nIn some cases, reference values may also be adjusted based on specific factors such as age, sex, or ethnicity. This is done to account for variations in normal values that may occur in different populations.\n\nIt is important to note that reference values are not set in stone and may vary slightly between different laboratories and testing methods. Additionally, reference values are meant to be used as a general guide and should be interpreted in conjunction with other clinical information to make an accurate diagnosis.'

In [45]:
import torch
from transformers import AutoTokenizer, AutoModel

In [46]:
# Load pre-trained BlueBERT model and tokenizer
model_name = "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [47]:

# Sample conversation
conversation = """
Patient: Hi, I’ve been feeling unwell for the past few days.
Patient: I’ve had a fever for the past three days, along with a sore throat and body aches.
Patient: I do have a mild cough, and my throat feels dry. But I don’t have any nausea or difficulty breathing.
Patient: No, I haven’t traveled, but a few of my colleagues at work had a cold last week.
Patient: No, I don’t have any chronic illnesses, and I’m not allergic to anything.
Patient: I took a paracetamol yesterday for the fever, and I’ve been drinking warm water and resting.
"""

In [48]:
# Tokenize the conversation
inputs = tokenizer(conversation, return_tensors="pt", truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [49]:
# Get model embeddings
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_length, hidden_size)

In [50]:
embeddings

tensor([[[ 0.6700,  0.0529, -0.0106,  ...,  0.0439,  0.3718,  0.1110],
         [-0.8095,  0.0119, -0.5421,  ..., -0.1386,  0.1275, -0.2058],
         [-0.3707,  0.1895,  0.0814,  ..., -0.3273,  0.5411,  0.0334],
         ...,
         [ 0.5727,  0.7360,  0.2596,  ...,  0.3456,  0.1816, -0.1227],
         [-0.0589,  0.0079,  0.2925,  ...,  0.2673,  0.8035, -0.5467],
         [ 0.9751,  0.1525, -0.1767,  ..., -0.2151,  0.5104, -0.0517]]])

In [51]:
keywords = {
    "symptoms": ["fever", "sore throat", "body aches", "cough", "headache", "nausea", "difficulty breathing"],
    "exposure": ["contact", "sick", "colleagues", "traveled"],
    "medical_history": ["diabetes", "high blood pressure", "asthma", "allergies", "chronic illness"],
    "medications": ["paracetamol", "ibuprofen", "antibiotic"]
}

In [52]:
# Extract detected keywords
extracted_info = {key: [] for key in keywords}
for key, values in keywords.items():
    for word in values:
        if word in conversation.lower():
            extracted_info[key].append(word)

# Print extracted details
print("Extracted Medical Information:")
for category, details in extracted_info.items():
    print(f"{category.capitalize()}: {', '.join(details) if details else 'None found'}")


Extracted Medical Information:
Symptoms: fever, sore throat, body aches, cough, nausea, difficulty breathing
Exposure: colleagues, traveled
Medical_history: chronic illness
Medications: paracetamol


Langchain Based Approach for not hardcoding the categories

In [53]:
%pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [54]:
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
import json

In [ ]:
# Set OpenAI API Key (Replace with your actual API key)
os.environ["OPENAI_API_KEY"] = ""

In [56]:
# Load OpenAI model via LangChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [57]:
# Load BlueBERT model and tokenizer
model_name = "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [58]:
# Function to extract medical information using LangChain
def extract_medical_info(conversation):
    prompt = f"""
    Extract relevant medical information from the following conversation.
    Categorize the extracted details under:
    - Symptoms
    - Exposure
    - Medical History
    - Medications

    Only return structured JSON format without extra explanations.

    Conversation:
    \"\"\"{conversation}\"\"\"

    Output:
    {{
        "symptoms": [],
        "exposure": [],
        "medical_history": [],
        "medications": []
    }}
    """

    messages = [
        SystemMessage(content="You are an AI assistant specialized in medical text processing."),
        HumanMessage(content=prompt)
    ]

    response = llm(messages)

    # Parse JSON output
    extracted_info = json.loads(response.content)
    return extracted_info

In [59]:
# Function to generate BlueBERT embeddings
def get_bluebert_embedding(text):
    if not text:
        return None  # Return None if there's no relevant information in a category

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the sentence-level embedding (mean of last hidden state)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


In [60]:
# Step 1: Extract Medical Information Using LangChain
extracted_info = extract_medical_info(conversation)

# Step 2: Generate BlueBERT embeddings for each category
embeddings = {category: get_bluebert_embedding(" ".join(terms)) for category, terms in extracted_info.items() if terms}

# Print extracted info and corresponding embeddings
print("\nExtracted Medical Information:")
print(json.dumps(extracted_info, indent=2))

print("\nGenerated BlueBERT Embeddings:")
for category, embedding in embeddings.items():
    print(f"{category.capitalize()} Embedding Shape: {embedding.shape if embedding is not None else 'No Data'}")

<ipython-input-58-5f91abaf7f2f>:30: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Extracted Medical Information:
{
  "symptoms": [
    "fever",
    "sore throat",
    "body aches",
    "mild cough",
    "dry throat"
  ],
  "exposure": [
    "colleagues at work had a cold last week"
  ],
  "medical_history": [
    "no chronic illnesses",
    "not allergic to anything"
  ],
  "medications": [
    "paracetamol"
  ]
}

Generated BlueBERT Embeddings:
Symptoms Embedding Shape: (768,)
Exposure Embedding Shape: (768,)
Medical_history Embedding Shape: (768,)
Medications Embedding Shape: (768,)


In [64]:
# embeddings

In [71]:
import torch

# Load saved conversation embeddings
conversation_data = torch.load("/kaggle/working/conversation_embeddings.pt")

# Convert conversation embeddings to (1, 768) shape
for category, embedding in conversation_data["embeddings"].items():
    if isinstance(embedding, torch.Tensor) and len(embedding.shape) == 1:  # Ensure it's a 1D tensor (768,)
        conversation_data["embeddings"][category] = embedding.unsqueeze(0)  # Convert to (1, 768)

# Verify changes
for category, embedding in conversation_data["embeddings"].items():
    print(f"{category} Embedding Shape: {embedding.shape}")  # Should now be (1, 768)

# Save back the modified embeddings if needed
torch.save(conversation_data, "/kaggle/working/conversation_embeddings.pt")

symptoms Embedding Shape: torch.Size([1, 768])
exposure Embedding Shape: torch.Size([1, 768])
medical_history Embedding Shape: torch.Size([1, 768])
medications Embedding Shape: torch.Size([1, 768])


<ipython-input-71-f4f4bb303d32>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  conversation_data = torch.load("/kaggle/working/conversation_embeddings.pt")


In [73]:
torch.save(conversation_data, "/kaggle/working/conversation_embeddings_updated.pt")
print("Updated conversation embeddings saved successfully!")

Updated conversation embeddings saved successfully!


In [74]:
# Load the updated embeddings
conversation_data_loaded = torch.load("/kaggle/working/conversation_embeddings_updated.pt")

# Print the shapes to verify they are still (1, 768)
for category, embedding in conversation_data_loaded["embeddings"].items():
    print(f"{category} Embedding Shape: {embedding.shape}")  # Should be (1, 768)

symptoms Embedding Shape: torch.Size([1, 768])
exposure Embedding Shape: torch.Size([1, 768])
medical_history Embedding Shape: torch.Size([1, 768])
medications Embedding Shape: torch.Size([1, 768])


<ipython-input-74-308e89dd1ed5>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  conversation_data_loaded = torch.load("/kaggle/working/conversation_embeddings_updated.pt")


In [72]:
for category, embedding in embeddings.items():
    print(f"{category.capitalize()} Embedding Shape: {embedding.shape if embedding is not None else 'No Data'}")

Symptoms Embedding Shape: (768,)
Exposure Embedding Shape: (768,)
Medical_history Embedding Shape: (768,)
Medications Embedding Shape: (768,)


In [63]:
import torch
import pickle
import json
import numpy as np

# Step 1: Extract Medical Information Using LangChain
extracted_info = extract_medical_info(conversation)

# Step 2: Generate BlueBERT embeddings for each category
embeddings = {category: get_bluebert_embedding(" ".join(terms)) for category, terms in extracted_info.items() if terms}

# Convert NumPy arrays to PyTorch tensors before saving
conversation_data = {
    "extracted_info": extracted_info,  # Raw extracted medical data
    "embeddings": {category: torch.tensor(embedding) if isinstance(embedding, np.ndarray) else embedding for category, embedding in embeddings.items() if embedding is not None}
}

# Save as a PyTorch .pt file
torch.save(conversation_data, "conversation_embeddings.pt")

# Save as a Pickle .pkl file
with open("conversation_embeddings.pkl", "wb") as f:
    pickle.dump(conversation_data, f)

print("Conversation embeddings saved successfully!")


Conversation embeddings saved successfully!
